### CNN on CIFR

1.  You cannot use DropOut layers.
2.  You MUST use Image Augmentation Techniques.
3.  You cannot use an already trained model as a beginning points, you have to initilize as your own
4.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
5.  You cannot use test images for training the model.
6.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
7.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
8. You cannot have more than 1 Million parameters in total
9. You can use any optimization algorithm you need. 
10. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
import tensorflow as tf

In [ ]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 100
l = 12
num_filter = 58
compression = 1
dropout_rate = 0.2

In [ ]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [ ]:
print(X_train.shape)
print(X_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Image data augmentation is typically only applied to the training dataset, and not to the validation or test dataset
datagen = ImageDataGenerator(rescale=1.0/255.0,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True, 
                             shear_range=0.2,zoom_range=0.2,validation_split=0.2)
train_iterator = datagen.flow(X_train, y_train)

#Image pixel scaling must be performed consistently across all datasets that interact with the model.
datagen = ImageDataGenerator(rescale=1.0/255.0)
test_iterator = datagen.flow(X_test, y_test)

In [ ]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (5,5), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

# Transition Block
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (5,5), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

# Output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [ ]:
num_filter = 12
dropout_rate = 0
l = 10
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [ ]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_132 (Conv2D)            (None, 32, 32, 12)   324         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_132 (Batch  (None, 32, 32, 12)  48          ['conv2d_132[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_132 (Activation)    (None, 32, 32, 12)   0           ['batch_normalization_132[0

In [ ]:
print(len(model.layers))

179


In [ ]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [ ]:
class Callback(tf.keras.callbacks.Callback):
	def on_epoch_end(self, epoch, logs=None):
		if(logs.get('accuracy') > 0.90):
			self.model.stop_training = True

In [ ]:
callback = Callback()

In [ ]:
model.fit(train_iterator, steps_per_epoch=(len(train_iterator)//2), epochs=100,callbacks=[callback])

Epoch 1/100
781/781 [==============================] - 40s 51ms/step - loss: 1.3995 - accuracy: 0.4925
Epoch 2/100
781/781 [==============================] - 40s 51ms/step - loss: 1.2594 - accuracy: 0.5413
Epoch 3/100
781/781 [==============================] - 40s 51ms/step - loss: 1.1418 - accuracy: 0.5940
Epoch 4/100
781/781 [==============================] - 40s 51ms/step - loss: 1.0633 - accuracy: 0.6231
Epoch 5/100
781/781 [==============================] - 40s 51ms/step - loss: 0.9775 - accuracy: 0.6553
Epoch 6/100
781/781 [==============================] - 40s 51ms/step - loss: 0.9250 - accuracy: 0.6749
Epoch 7/100
781/781 [==============================] - 40s 51ms/step - loss: 0.8538 - accuracy: 0.6996
Epoch 8/100
781/781 [==============================] - 40s 51ms/step - loss: 0.8193 - accuracy: 0.7115
Epoch 9/100
781/781 [==============================] - 41s 52ms/step - loss: 0.7847 - accuracy: 0.7240
Epoch 10/100
781/781 [==============================] - 40s 52ms/step - l

In [ ]:
# Test the model
loss, acc = model.evaluate(test_iterator, steps=(len(test_iterator)//batch_size))
print('Test loss:', loss)
print('Test Accuracy: %.3f' % (acc * 100))

4/4 [==============================] - 0s 19ms/step - loss: 0.2439 - accuracy: 0.9375
Test loss: 0.24391716718673706
Test Accuracy: 93.750


In [ ]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


**REFERENCES:**
1. https://machinelearningmastery.com/how-to-normalize-center-and-standardize-images-with-the-imagedatagenerator-in-keras/
2. https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/
3. https://machinelearningmastery.com/check-point-deep-learning-models-keras/